In [ ]:
import json
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

In [ ]:
def loadjson(fn):
    f = open(fn)
    buf = f.read()
    f.close()
    
    obj = json.loads(buf)
    return obj

# 分析消息头占比，且消息头修改后大小变化
def analyzeSenderMsgHead(inobj, msgheadsize2):
    ret = {
        'totalbytes': 0,
        'totalmsgs': 0,
        'totalheadbytes': 0,
        'totalheadbytes2': 0,        
    }  
    
    for msgid in inobj['mapMsgs']:
        ret['totalbytes'] += inobj['mapMsgs'][msgid]['totalBytes']
        ret['totalmsgs'] += inobj['mapMsgs'][msgid]['totalTimes']
        ret['totalheadbytes'] += 12 * inobj['mapMsgs'][msgid]['totalTimes']
        ret['totalheadbytes2'] += msgheadsize2 * inobj['mapMsgs'][msgid]['totalTimes']
        
    return ret

def formatBytes(nums):
    if nums < 1024:
        return '{}'.format(nums)
    
    if nums < 1024 * 1024:
        return '{}k'.format(nums / 1024)
    
    if nums < 1024 * 1024 * 1024:
        return '{}m'.format(nums / 1024 / 1024)
    

    return '{}g'.format(nums / 1024 / 1024 / 1024)    

In [ ]:
obj = loadjson("./gateway.sender.1650034208.json")
msg1 = analyzeSenderMsgHead(obj, 8)
msg2 = analyzeSenderMsgHead(obj, 4)

print('一共发送消息数为：', msg1['totalmsgs'])
print('共消耗：', formatBytes(msg1['totalbytes']))
print('平均消息大小为：', msg1['totalbytes'] / msg1['totalmsgs'])
print('消息头占比为：', msg1['totalheadbytes'] / msg1['totalbytes'])
print('如果消息头减小4字节，整体带宽将减少到：', (msg1['totalbytes'] - msg1['totalheadbytes'] + msg1['totalheadbytes2']) / msg1['totalbytes'])
print('如果消息头减小8字节，整体带宽将减少到：', (msg2['totalbytes'] - msg2['totalheadbytes'] + msg2['totalheadbytes2']) / msg2['totalbytes'])

In [ ]:
# 看协议整体消息大小占比
def showTotalMsgSize(inobj, maxsize = 10):
    lstmsgid = []
    lstmsgsize = []
    nums = 0
    
    for msgid in inobj['mapMsgs']:
        lstmsgid.append(msgid)
        lstmsgsize.append(inobj['mapMsgs'][msgid]['totalBytes'])
        
        nums += 1
        if nums >= maxsize:
            break
        
    fig = go.Figure(data=[go.Pie(labels=lstmsgid, values=lstmsgsize)])
    fig.update_layout(title_text='协议整体占比（含次数）')
    fig.show()

In [ ]:
showTotalMsgSize(obj)

In [ ]:
# 看协议平均大小占比
def showAvgMsgSize(inobj, maxsize = 10):
    lstmsgid = []
    lstmsgsize = []
    nums = 0
    
    for msgid in inobj['mapMsgs']:
        lstmsgid.append(msgid)
        lstmsgsize.append(inobj['mapMsgs'][msgid]['totalBytes'] / inobj['mapMsgs'][msgid]['totalTimes'])
        
        nums += 1
        if nums >= maxsize:
            break
        
    fig = go.Figure(data=[go.Pie(labels=lstmsgid, values=lstmsgsize)])
    fig.update_layout(title_text='协议平均大小占比')
    fig.show()

In [ ]:
showAvgMsgSize(obj)

In [ ]:
# 看协议次数占比
def showMsgTimes(inobj, maxsize = 10):
    lstmsgid = []
    lstmsgsize = []
    nums = 0
    
    for msgid in inobj['mapMsgs']:
        lstmsgid.append(msgid)
        lstmsgsize.append(inobj['mapMsgs'][msgid]['totalTimes'])
        
        nums += 1
        if nums >= maxsize:
            break
        
    fig = go.Figure(data=[go.Pie(labels=lstmsgid, values=lstmsgsize)])
    fig.update_layout(title_text='协议次数占比')
    fig.show()

In [ ]:
showMsgTimes(obj)